In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import glob

hives_ids = ["smrpiclient7", "smrpiclient6", "smrpiclient3"]

In [2]:
import matplotlib.pyplot as plt

%matplotlib widget

def plot_spectrogram(frequency, time_x, spectrocgram, title):
    fig = plt.figure(figsize=(6,4))
    plt.title(title)
    plt.pcolormesh(time_x, frequency, spectrocgram)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()

In [8]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 1 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 1, 2, stride=2)


    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # compressed representation
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
        return x

In [ ]:
import torch
from torch import nn

class autoencoder_basic(nn.Module):
    def __init__(self):
        super(autoencoder_basic, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1499, 512),
            nn.SELU(True),
            nn.Linear(512, 128),
            nn.SELU(True),
            nn.Linear(128, 64),
            nn.SELU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.SELU(True),
            nn.Linear(128, 512),
            nn.SELU(True),
            nn.Linear(128, 1499),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


## Load train data samples

In [3]:
import struct
import math
import numpy as np
import librosa
import librosa.display

from tqdm import tqdm
from datetime import datetime
from scipy.io import wavfile
from scipy import signal
from scipy.fftpack import fft

import matplotlib.pyplot as plt
%matplotlib widget

sound_time_ms = 2000
# ~93 ms for fft window
nfft = 4096
# ~34% overlapping
hop_len = (nfft//3) + 30
# This can be manipulated to adjust number of bins for conv layer
fmax = 2750

hives_data = []
max_to_norm = 0
for idx, hive_id in enumerate(hives_ids):
    sound_files = [f for f in glob.glob(f"..\\measurements\\smartulav2\\{hive_id}_*\\*.wav")]
    print(f"Sound data preparation for hive: {hive_id} which has {len(sound_files)} recordings...", end=' ', flush=True)
    for file in tqdm(sound_files):
        sample_rate, sound_samples = wavfile.read(file)
        sound_samples = sound_samples.T[0]/(2.0**31)
    
        spectrogram = librosa.core.stft(sound_samples, n_fft=nfft, hop_length=hop_len)
        spectrogram_magnitude = np.abs(spectrogram)
        spectrogram_phase = np.angle(spectrogram)
        spectrogram_db = librosa.amplitude_to_db(spectrogram_magnitude, ref=np.max)
        frequencies = librosa.fft_frequencies(sr=sample_rate, n_fft=nfft)
        times = (np.arange(0, spectrogram_magnitude.shape[1])*hop_len)/sample_rate
        
        freq_slice = np.where((frequencies < fmax))
        frequencies = frequencies[freq_slice]
        spectrogram_db = spectrogram_db[freq_slice, :][0]    
    
        filename = file.rsplit('\\', 1)[-1]
        timestamp = filename[filename.index('-')+1:].rsplit(".wav")[0]
        datetime = datetime.strptime(timestamp, '%Y-%m-%dT%H-%M-%S')
        hives_data.append([datetime, hive_id, sound_samples, [frequencies, times, spectrogram_db]])
    print(" done.")

print(f"Got {len(hives_data)} sound samples")

Sound data preparation for hive: smrpiclient7 which has 3367 recordings... 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3367/3367 [00:41<00:00, 80.29it/s]

 done.
Sound data preparation for hive: smrpiclient6 which has 3172 recordings... 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3172/3172 [00:40<00:00, 77.83it/s]

 done.
Sound data preparation for hive: smrpiclient3 which has 602 recordings... 


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 602/602 [00:08<00:00, 73.90it/s]

 done.
Got 7141 sound samples


## Prepare data for autoencoder train.

In [ ]:
import random 

random_idx = random.randint(0, len(hives_data) - 1)

plot_spectrogram(hives_data[random_idx][3][0],
                 hives_data[random_idx][3][1],
                 hives_data[random_idx][3][2],
                 f"hive: {hives_data[random_idx][1]}, time: {hives_data[random_idx][0]}") 

In [ ]:
import librosa
import pandas as pd

from scipy import signal as sig
from sklearn.preprocessing import MinMaxScaler
                      
pd_data = pd.DataFrame(
    {
        'samples': [hive_data[2] for hive_data in hives_data],
        'periodogram' : [hive_data[3] for hive_data in hives_data],
        'sn': [hive_data[1] for hive_data in hives_data]
    }
)

pd_data['periodogram'] = list(np.transpose(MinMaxScaler().fit_transform(np.transpose(pd_data['periodogram'].values.tolist()))))
# pd_data['mfcc'] = list([np.mean(librosa.feature.mfcc(sample, sr=3000, n_fft=150, hop_length=75, n_mfcc=14), axis=1)
#                            for sample in tpd_data['samples'].to_numpy()])
# pd_ae_data = pd_data[(pd_data['sn'] == 'smrpiclient7')]
pd_ae_data = pd_data
# pd_ae_data = pd_data[(pd_data['sn'] == 1300001)
#                         | (pd_data['sn'] == 1400001)
#                         | (pd_data['sn'] == 1400002)]
pd_ae_data = pd_ae_data.sample(frac=1).reset_index(drop=True)

## Train basic AE

In [ ]:
from scipy import signal as sig
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from torch.utils import data as tdata

# Divide data to test, validation and train
train_stop_idx = int(pd_ae_data.shape[0]*90/100)

pd_ae_data_train = pd_ae_data[:train_stop_idx]
pd_ae_data_test = pd_ae_data[train_stop_idx:]

print(f'Train data size: {pd_ae_data_train.shape[0]}')
print(f'Test data size: {pd_ae_data_test.shape[0]}')

tensor_train = torch.Tensor(pd_ae_data_train['periodogram'].values.tolist())
tensor_test = torch.Tensor(pd_ae_data_test['periodogram'].values.tolist())

train_dataset = tdata.TensorDataset(tensor_train)
test_dataset = tdata.TensorDataset(tensor_test)

dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.autograd import Variable
from torchvision import transforms
import torch.nn.functional as F

num_epochs = 1000
learning_rate = 1e-3

model = autoencoder_basic().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    loss = 0
    for data in dataloader:
        periodogram = data[0].to(device)
        # ===================forward=====================
        output = model(periodogram)
        #train_loss = criterion(output, periodogram)
        train_loss = F.binary_cross_entropy(output, periodogram)
        # ===================backward====================
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()
    loss = loss / len(dataloader)
    if (epoch+1) % (num_epochs/10) == 0:
        print(f'epoch {epoch + 1}/{num_epochs}, loss:{loss}')

## Train CONV AE

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# sc = StandardScaler()
mm = MinMaxScaler()

spectrogram_ae_data = [hive_data[3][2] for hive_data in hives_data]
# standarized_ae_data = [sc.fit_transform(spec.T).T for spec in conv_ae_data]
scaled_ae_data = [mm.fit_transform(stan.T).T for stan in spectrogram_ae_data]

print(f"Got dataset of size: {len(scaled_ae_data)}")

Got dataset of size: 7141


In [12]:
import torch 
from torch.utils import data as tdata

data_ratio = int(len(scaled_ae_data)*90/100)

dataset_tensor = torch.Tensor(scaled_ae_data)
print(f"Dataset shape: {dataset_tensor.shape}")
# add one extra dimension as it is required for conv layer
dataset_tensor = dataset_tensor[:, None, :, :] 
dataset = tdata.TensorDataset(dataset_tensor)
train_set, test_set = torch.utils.data.random_split(dataset, [data_ratio, len(scaled_ae_data)-data_ratio])

dataloader_train = tdata.DataLoader(train_set, batch_size=32, shuffle=True)
dataloader_test = tdata.DataLoader(test_set, batch_size=32, shuffle=True)

Dataset shape: torch.Size([7141, 256, 64])


In [16]:
num_epochs = 10
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

modelConvAE = ConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(modelConvAE.parameters(), lr=learning_rate, weight_decay=1e-5)

print(modelConvAE)

ConvAutoencoder(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(4, 16, kernel_size=(2, 2), stride=(2, 2))
  (t_conv2): ConvTranspose2d(16, 1, kernel_size=(2, 2), stride=(2, 2))
)


In [18]:
for epoch in range(1, num_epochs+1):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data in dataloader_train:
        # _ stands in for labels, here
        # no need to flatten images
        periodogram = data[0].to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = modelConvAE(periodogram)
        # calculate the loss
        loss = criterion(outputs, periodogram)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*periodogram.size(0)
    # print avg training statistics 
    train_loss = train_loss/len(dataloader_train)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

Epoch: 1 	Training Loss: 1.098176
Epoch: 2 	Training Loss: 1.014356
Epoch: 3 	Training Loss: 0.937788
Epoch: 4 	Training Loss: 0.914127
Epoch: 5 	Training Loss: 0.908649
Epoch: 6 	Training Loss: 0.872004
Epoch: 7 	Training Loss: 0.846211
Epoch: 8 	Training Loss: 0.840525
Epoch: 9 	Training Loss: 0.836914
Epoch: 10 	Training Loss: 0.833600


In [ ]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

# convert data to torch.FloatTensor
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='../MNIST-data', train=True, download=True, transform=transform)
dataloader_train = torch.utils.data.DataLoader(train_data, batch_size=20, num_workers=0)

In [ ]:
for idx, data in enumerate(train_loader):
    print((data[0]).shape)
    break

## Test

In [ ]:
import matplotlib.pyplot as plt

counter = 0
with torch.no_grad():
    loss_test = 0
    for data in dataloader_test:
        periodograms_test = data[0].to(device)
        output = model(periodograms_test)
        for idx, i in enumerate(output):
            #loss_test += nn.MSELoss()(periodograms_test[idx], i)
            loss_test += F.binary_cross_entropy(periodograms_test[idx], i)

loss_test = loss_test/len(pd_ae_data_test)
print(f'Final test loss: {loss_test}')


Plot and visualize (MFCC vs AE)

In [ ]:
import matplotlib.pyplot as plt

with torch.no_grad():
    encode_data = pd_ae_data['periodogram'].values.tolist()
    encode_data_tensor = torch.Tensor(encode_data).to(device)
    output = [model.encoder(encode_data_tensor).cpu().numpy()][0]


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

idx =708
with torch.no_grad():
    fig = plt.figure()
    plt.figure(figsize=(8,4))
    plt.plot(encode_data[idx], 'r')
    plt.plot(model.decoder(torch.Tensor(output[idx]).to(device)).cpu().numpy(), 'b')

Dimension reduction - now we perform t-SNE and PCA to visualize the data

In [ ]:
from sklearn.manifold import TSNE

reduced_ae_tsne = TSNE(n_components=2, perplexity=100, learning_rate=500, verbose=1).fit_transform(output)
reduced_mfcc_tsne = TSNE(n_components=2, perplexity=100, learning_rate=500, verbose=1).fit_transform(pd_ae_data['mfcc'].values.tolist())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

mfccs_standarized = StandardScaler().fit_transform(pd_ae_data['mfcc'].values.tolist())
ae_standarized = StandardScaler().fit_transform(output)
reduced_ae_pca = PCA(n_components = 2).fit_transform(ae_standarized)
reduced_mfcc_pca = PCA(n_components = 2).fit_transform(mfccs_standarized)

In [ ]:
import matplotlib.pyplot as plt

pca = PCA().fit(ae_standarized)

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.005), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=43, verbose=0, behaviour="new")

clf.fit(reduced_ae_pca)
clf.get_params(deep=True)

In [ ]:
output_forest = clf.predict(reduced_ae_pca)

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(10,8))
fig.subplots_adjust(hspace=0.4)

idx_1300001 = pd_data[pd_data['sn'] == 1300001].index.values.tolist()
idx_1300002 = pd_data[pd_data['sn'] == 1300002].index.values.tolist()
idx_1400001 = pd_data[pd_data['sn'] == 1400001].index.values.tolist()
idx_1400002 = pd_data[pd_data['sn'] == 1400002].index.values.tolist()

l1_ae_tsne = axs[0][0].scatter([data[0] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1300001],
                 [data[1] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1300001], c='r', alpha = 0.3)
# l2_ae_tsne = axs[0][0].scatter([data[0] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1300002],
#                  [data[1] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1300002], c='b', alpha = 0.3)
# l3_ae_tsne = axs[0][0].scatter([data[0] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1400001],
#                  [data[1] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1400001], c='g', alpha = 0.3)
# l4_ae_tsne = axs[0][0].scatter([data[0] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1400002],
#                  [data[1] for idx, data in enumerate(reduced_ae_tsne) if idx in idx_1400002], c='y', alpha = 0.3)

l1_mfcc_tsne = axs[0][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1300001],
                 [data[1] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1300001], c='r', alpha = 0.3)
# l2_mfcc_tsne = axs[0][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1300002],
#                  [data[1] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1300002], c='b', alpha = 0.3)
# l3_mfcc_tsne = axs[0][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1400001],
#                  [data[1] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1400001], c='g', alpha = 0.3)
# l4_mfcc_tsne = axs[0][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1400002],
#                  [data[1] for idx, data in enumerate(reduced_mfcc_tsne) if idx in idx_1400002], c='y', alpha = 0.3)

# axs[0][0].legend((l1_ae_tsne, l2_ae_tsne, l3_ae_tsne, l4_ae_tsne), ('Sikorki', 'Not Hive', 'Sulmin 1', 'Sulmin 2'),
#                  loc='upper right')
axs[0][0].set_xlabel('DIM1')
axs[0][0].set_ylabel('DIM2')
axs[0][0].set_title('Hives - AE sounds with TSNE')

# axs[0][1].legend((l1_mfcc_tsne, l2_mfcc_tsne, l3_mfcc_tsne, l4_mfcc_tsne), ('Sikorki', 'Not Hive', 'Sulmin 1', 'Sulmin 2'),
#                  loc='upper right')
axs[0][1].set_xlabel('DIM1')
axs[0][1].set_ylabel('DIM2')
axs[0][1].set_title('Hives - MFCC sounds with TSNE')

l1_ae_pca = axs[1][0].scatter([data[0] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1300001],
                 [data[1] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1300001], c='r', alpha = 0.3)
markers = axs[1][0].scatter([data[0] for idx, data in enumerate(reduced_ae_pca) if output_forest[idx] == -1],
                           [data[1] for idx, data in enumerate(reduced_ae_pca) if output_forest[idx] == -1], c='b', alpha = 0.9)
# l2_ae_pca = axs[1][0].scatter([data[0] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1300002],
#                  [data[1] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1300002], c='b', alpha = 0.3)
# l3_ae_pca = axs[1][0].scatter([data[0] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1400001],
#                  [data[1] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1400001], c='g', alpha = 0.3)
# l4_ae_pca = axs[1][0].scatter([data[0] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1400002],
#                  [data[1] for idx, data in enumerate(reduced_ae_pca) if idx in idx_1400002], c='y', alpha = 0.3)

l1_mfcc_pca = axs[1][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1300001],
                 [data[1] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1300001], c='r', alpha = 0.3)
# l2_mfcc_pca = axs[1][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1300002],
#                  [data[1] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1300002], c='b', alpha = 0.3)
# l3_mfcc_pca = axs[1][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1400001],
#                  [data[1] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1400001], c='g', alpha = 0.3)
# l4_mfcc_pca = axs[1][1].scatter([data[0] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1400002],
#                  [data[1] for idx, data in enumerate(reduced_mfcc_pca) if idx in idx_1400002], c='y', alpha = 0.3)

# axs[1][0].legend((l1_ae_pca, l2_ae_pca, l3_ae_pca, l4_ae_pca), ('Sikorki', 'Not Hive', 'Sulmin 1', 'Sulmin 2'),
#                  loc='upper right')
axs[1][0].set_xlabel('PC1')
axs[1][0].set_ylabel('PC2')
axs[1][0].set_title('Hives - AE sounds with PCA')

# axs[1][1].legend((l1_mfcc_pca, l2_mfcc_pca, l3_mfcc_pca, l4_mfcc_pca), ('Sikorki', 'Not Hive', 'Sulmin 1', 'Sulmin 2'),
#                   loc='upper right')
axs[1][1].set_xlabel('PC1')
axs[1][1].set_ylabel('PC2')
axs[1][1].set_title('Hives - MFCC sounds with PCA')



In [ ]:
plt.savefig('output-ae/basic-2warstwy-128-32.png')

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft, fftfreq

RECORD_TIME = 2
SAMPLE_RATE = 44100

data = hives_data[-10][2][:, 0]/(2.0**31)
datetime = hives_data[-1][0]
fft_data = abs(fft(data))
freqs = fftfreq(int(len(fft_data)/2), 1/SAMPLE_RATE)

fig, axs = plt.subplots(2)
fig.tight_layout(pad=3.0)
axs[0].set_title(f"Sound recording at {datetime} ({RECORD_TIME}s)")
axs[0].grid()
axs[0].set_xlabel('Time [sec]')
axs[0].plot(np.linspace(0, 2, len(data)), data)

axs[1].set_title("Periodogram")
axs[1].set_xticks(np.arange(0, (freqs.size/2), step=100))
axs[1].set_xticklabels(np.arange(0, (freqs.size/2), step=100, dtype=int), rotation=45)
axs[1].grid()
axs[1].set_xlabel('Frequency [Hz]')
axs[1].plot(freqs[1:1500], fft_data[1:1500], 'r')